
We show how to solve a capacitated facility location problem (CFLP) in Julia using a MILP-Solver and how to use solver dependent callbacks to implement a custom branching and selection rule for GLPK.

In order to keep our example simple we choosed a classical branching rule - namely "branch on the most fractional variable" - which is often part of a MILP-Solver-implementation.

## Problem statement - capacitated facility location problem

A company must select a subset of potential facility locations to minimize total cost associated with opening facilities and servicing customers. Each costumer has a specific demand and each facility has a fixed opening costs, a specific capacity and service costs based on the distance to customers.


## Mathematical model

First we list the the given information before we write down the model in a more mathematical way 

### Sets

- $C=\{1,\ldots,n\}$ of clients
- $J=\{1,\ldots,m\}$ of potential facilities

### Parameter

- $c_j^f$ fixed costs for opening facility $j$
- $c_{ij}^v$ variable costs for transporting goods from facility $j$ to client $i$
- $d_i$ demand of client $i$
- $q_j$ facility capacity

### Decision variables

- $y_i$, binary, 1 iff facility $j$ is used
- $x_{ij}$, real, demand of client $i$ which is served by facility $j$

### Objective

Minimize the sum of fix and variable costs:

$$
\min \sum_j c^f_j y_j + \sum_{ij} c_{ij}^v x_{ij}
$$

### Constraints

- (c1) Each client's demand is served:

$$
\sum_j x_{ij} = d_i, \; \forall i \in C
$$

- (c2) A facility can only serve a client if its open:

$$
x_{ij} \leq d_i y_j, \forall i\in C, \forall j \in J
$$

- (c3) Capacity constraint

$$
\sum_i x_{ij} \leq q_j y_j, \forall j \in J
$$

#### MILP Formulation

$$
\begin{array}{lll}
\min & \sum_j c_j^f\cdot y_j + \sum_{ij} c_{ij}^v \cdot x_{ij} &\\
s.t. & \sum_j x_{ij} = d_i, & \forall i \in C \\
     & x_{ij} \leq d_i y_j, & \forall i \in C, \forall j \in J \\
     & \sum_i x_{ij} \leq q_j y_j & \forall j\in J \\
     & y_j \in \{ 0,1 \}, & \forall j\in J\\
     & x_{ij} \in [0,d_i], & \forall i\in C, \forall j\in J
\end{array}
$$

## A simple branching and selection strategy

For the simplicity of the example we implement the following branching/selection strategy:
1. branch on the most fractional variable

$$\arg\max \{|y_j - 0.5| \forall j \in J\}$$
   
3. and select
   1. if $y_j - 0.5 < 0$: down-branch and
   1. if $y_j - 0.5 > 0$:  up-branch.[^1]


### Remarks on branching and selection

1. We remark that using the most fractional value as branching criteria is a common technique, which is also often implemented in solver. But we precisely choosed the criteria to on the one hand keep the example simple and one the other hand be able to test our implementation against existing ones.
2. There is no deeper reasoning behind the selection rule and we have choosen it just to keep the example simple.
3. In order to modify the example, you need to know the options for GLPK. They are comprehensively documented in the [PDF documentation](https://github.com/jump-dev/GLPK.jl/files/11143880/glpk.pdf).

[^1]: for the precise definition of up- and down-branch see section 5.2.9 in the [GLPK documentation](https://github.com/jump-dev/GLPK.jl/files/11143880/glpk.pdf)

In [1]:
using JuMP
import GLPK
import Random
import MathOptInterface as MOI

using JLD2

## Julia implementation

Before we look at an implemention of above model using Jump, let me briefly give you an overview on the full implementation

1. A data structure holding the problem data
1. a function to generate random problem data (for example to test the code)
4. a simple output-function
3. a function implementing the MILP (incl. custom branch and selection strategy)
5. we modified the MILP to see when our custom branching and selection strategy was used.

### CFLP data structure

We defined a data Data Structures `cflp_data` to hold the problem data.

In [2]:
include("SA_DataSturctures.jl")

### IO 

We wrote two functions to simply generate random problem data and to visualize the found solution.

In [3]:
include("IO.jl")

print_solution (generic function with 2 methods)

## MILP model implementation with solver dependent callbacks

In [4]:
function solve_cflp(data::cflp_data)
    """
    solves uncapacitated facility location problem
    """

    # instanciate model and set optimizer
    model = Model()
    set_optimizer(model, GLPK.Optimizer)

    # vars
    @variable(model, y[1:data.m], Bin) #open facilities
    @variable(model, x[1:data.n, 1:data.m], lower_bound = 0) # demand of client i served by facility j

    # objective
    fix_cost = @expression(model, sum( data.cost_fix[j] * y[j] for j in 1:data.m))
    var_cost = @expression(model, sum( data.cost_var[i,j] * x[i,j] for i in 1:data.n for j in 1:data.m))
    @objective(model, Min, fix_cost + var_cost)

    # constraints
    ## fulfill demand
    @constraint(model, c1[i in 1:data.n], sum(x[i,j] for j in 1:data.m) == data.demand[i])
    ## clients are served by open facility only
    @constraint(model, c2[i in 1:data.n, j in 1:data.m], x[i,j] <= data.demand[i] * y[j])
    ## obey facility capacity
    @constraint(model, c3[j in 1:data.m], sum(x[i,j] for i in 1:data.n) <= data.capacity[j] * y[j])
    
    function callback_function(cb_data)
        # determine reason for calling the callback routine
        reason = GLPK.glp_ios_reason(cb_data.tree)
        # ignore reason unless request for branching
        if reason != GLPK.GLP_IBRANCH
            return
        end
        y_vals = callback_value.(Ref(cb_data), y)
        # determinine most fractional value
        most_frac = findmin([abs(y_j - 0.5) for y_j in y_vals])[2]
        # check if we can branch upon specifed variable
        can_branch = GLPK.glp_ios_can_branch(cb_data.tree, most_frac)
        if can_branch != 0 && (y_vals[most_frac] - 0.5 < 0.0)
            return GLPK.glp_ios_branch_upon(cb_data.tree, most_frac, GLPK.GLP_DN_BRNCH)
        elseif can_branch != 0 && (y_vals[most_frac] - 0.5 > 0.0)
            return GLPK.glp_ios_branch_upon(cb_data.tree, most_frac, GLPK.GLP_UP_BRNCH)
        else
            # leave decision to solver
            return
        end       
    end
    
    MOI.set(model, GLPK.CallbackFunction(), callback_function)
    
    optimize!(model)

    # test before return solution
    if !is_solved_and_feasible(model)
        return error("Solver did not found an optimal solution")
    end
        
    output = Dict(
        "objective value" => objective_value(model),
        "facilities" => value.(y),
        "assignment" => value.(x)
    )
    
    return output
end

solve_cflp (generic function with 1 method)

## Solve a simple CFLP

The following problem data for a CFLP is taken from [Mathematical Optimization: Solving Problems using SCIP and Python](https://scipbook.readthedocs.io/en/latest/flp.html).

Its so simple that one can easily derive an optimal solution by hand. Moreover its a good practice to test our implementation on well known instances.

In [5]:
# use simple example data from SCIP docs
d = [80, 270, 250, 160, 180];
c_f = [1000.,1000.,1000.];
c_v = [9. 6. 4.; 5. 4. 7.; 6. 3. 4.; 8. 5. 3.; 10. 8. 4.];
q = [500,500,500]
data = cflp_data(c_f,c_v,q,d)

cflp_data([1000.0, 1000.0, 1000.0], [9.0 6.0 4.0; 5.0 4.0 7.0; … ; 8.0 5.0 3.0; 10.0 8.0 4.0], [500, 500, 500], [80, 270, 250, 160, 180], 3, 5, 87.0, 11.0)

In [6]:
solution = solve_cflp(data)
print_solution(solution)

Total cost are:5370.000000000006
Open facilities:[2, 3]
Assignment:
customer   1 gets       80.000000 from facility   3
customer   2 gets      270.000000 from facility   2
customer   3 gets      230.000000 from facility   2
customer   3 gets       20.000000 from facility   3
customer   4 gets      160.000000 from facility   3
customer   5 gets      180.000000 from facility   3


## A more sophisticated CFLP

Unfortunately the above example was so simple, that GLPK did not used our custom branching and selection rule.
Hence we look at a more sophisticated example next, which we found while testing our implementation on randomly generated data.

To visualize that our custom branching and selection rule was applied we added some `println` commands in our callback function:
```
        if can_branch != 0 && (y_vals[most_frac] - 0.5 < 0.0)
            println("used down-branch")
            return GLPK.glp_ios_branch_upon(cb_data.tree, most_frac, GLPK.GLP_DN_BRNCH)
        elseif can_branch != 0 && (y_vals[most_frac] - 0.5 > 0.0)
            println("used up-branch")
            return GLPK.glp_ios_branch_upon(cb_data.tree, most_frac, GLPK.GLP_UP_BRNCH)
```

In [7]:
function solve_cflp_println(data::cflp_data)
    """
    solves uncapacitated facility location problem
    """
    # derive parameter from data, i.e. 
    n = length(data.demand) # number of clients n 
    m = length(data.cost_fix) # number of facilities m

    # instanciate model and set optimizer
    model = Model()
    set_optimizer(model, GLPK.Optimizer)

    # vars
    @variable(model, y[1:m], Bin) #open facilities
    @variable(model, x[1:n, 1:m], lower_bound = 0) # demand of client i served by facility j

    # objective
    fix_cost = @expression(model, sum( data.cost_fix[j] * y[j] for j in 1:m))
    var_cost = @expression(model, sum( data.cost_var[i,j] * x[i,j] for i in 1:n for j in 1:m))
    @objective(model, Min, fix_cost + var_cost)

    # constraints
    ## fulfill demand
    @constraint(model, c1[i in 1:n], sum(x[i,j] for j in 1:m) == data.demand[i])
    ## clients are served by open facility only
    @constraint(model, c2[i in 1:n, j in 1:m], x[i,j] <= data.demand[i] * y[j])
    ## obey facility capacity
    @constraint(model, c3[j in 1:m], sum(x[i,j] for i in 1:n) <= data.capacity[j] * y[j])
    
    function callback_function(cb_data)
        # determine reason for calling the callback routine
        reason = GLPK.glp_ios_reason(cb_data.tree)
        # ignore reason unless request for branching
        if reason != GLPK.GLP_IBRANCH
            return
        end
        y_vals = callback_value.(Ref(cb_data), y)
        # determinine most fractional value
        most_frac = findmin([abs(y_j - 0.5) for y_j in y_vals])[2]
        # check if we can branch upon specifed variable
        can_branch = GLPK.glp_ios_can_branch(cb_data.tree, most_frac)
        if can_branch != 0 && (y_vals[most_frac] - 0.5 < 0.0)
            println("used down-branch")
            return GLPK.glp_ios_branch_upon(cb_data.tree, most_frac, GLPK.GLP_DN_BRNCH)
        elseif can_branch != 0 && (y_vals[most_frac] - 0.5 > 0.0)
            println("used up-branch")
            return GLPK.glp_ios_branch_upon(cb_data.tree, most_frac, GLPK.GLP_UP_BRNCH)
        else
            # leave decision to solver
            return
        end       
    end
    
    MOI.set(model, GLPK.CallbackFunction(), callback_function)
    
    optimize!(model)

    # test before return solution
    if !is_solved_and_feasible(model)
        return error("Solver did not found an optimal solution")
    end
        
    output = Dict(
        "objective value" => objective_value(model),
        "facilities" => value.(y),
        "assignment" => value.(x)
    )
    
    return output
end

solve_cflp_println (generic function with 1 method)

For simplicity we just print the first 5 customer to facility allocations.

In [8]:
d = load("../data/data.jld2")["data"]
rng_data = cflp_data(d["cost_fix"], d["cost_var"], d["capacity"], d["demand"])
solution = solve_cflp_println(rng_data)
print_solution(solution, true)

used up-branch
used up-branch
used up-branch
used down-branch
used down-branch
used up-branch
Total cost are:14277.130000000001
Open facilities:[1, 3, 5, 6, 8, 9, 10, 11, 13]
Assignment:
customer   1 gets       18.000000 from facility   9
customer   2 gets       21.000000 from facility  13
customer   3 gets       12.000000 from facility  11
customer   4 gets       14.000000 from facility   3
customer   5 gets       18.000000 from facility  13


## Remarks on code improvements

We like to refer to two sources [to improve the performance](https://jump.dev/JuMP.jl/stable/tutorials/getting_started/performance_tips/) and [to improve design the design for larger models](https://jump.dev/JuMP.jl/stable/tutorials/getting_started/design_patterns_for_larger_models/).

In particular it would be benefitial to create a module in the long run, but as we wanted to show how to implement solver dependet callbacks in Julia, we decided not to do it.